## ゆっくり検索

おすすめスレをEmbeddingして読みたいジャンルに近いような作品を提示してくれるプログラムです。


プログラムの説明についてはChatGPTに書いてもらったもので正確さは保証しません。

In [ ]:
!pip install openai
!pip install llama-index
!pip install langchain
!pip install nltk
!pip install ipywidgets

## モジュールのインポート

このプログラムでは、最初に`os`モジュール、`json`モジュール、`openai`モジュールをインポートしています。
次に、`langchain.embeddings`モジュールから`OpenAIEmbeddings`クラス、`llama_index.llms`モジュールから`AzureOpenAI`クラスをインポートします。
また、`llama_index`モジュールから`LangchainEmbedding`クラス、`VectorStoreIndex`クラス、`SimpleDirectoryReader`クラス、`ServiceContext`クラスをインポートします。
さらに、`logging`モジュールと`sys`モジュールをインポートします。
`logging.basicConfig()`メソッドを使用して、ログの出力先を標準出力に設定し、ログのレベルを`logging.INFO`に設定しています。より詳細な出力が必要な場合は、`logging.DEBUG`を使用します。
最後に、`logging.getLogger().addHandler()`メソッドを使用して、ログハンドラーを設定しています。

In [1]:
import os
import json
import openai
from langchain.embeddings import OpenAIEmbeddings
from llama_index.llms import OpenAI #AzureOpenAI
from llama_index import LangchainEmbedding
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, SimpleWebPageReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## OpenAIの設定

`"../key.txt"`ファイルからAPIキーを読み取り、`openai.api_key`に設定します。さらに、環境変数`OPENAI_API_KEY`にもAPIキーを設定します。

次に、ChatGPT-35-turboモデルを使用するために、`OpenAI`クラスのインスタンス`llm`を作成します。

また、埋込みモデルとして`text-embedding-ada-002`モデルを使用するために、`OpenAIEmbeddings`クラスのインスタンスを作成し、`LangchainEmbedding`クラスのインスタンス`embedding_llm`を作成します。このとき、先ほどのAPIキーやAPIの設定を使用します。

`ServiceContext.from_defaults()`メソッドを使用して、`llm`と`embedding_llm`を含む`ServiceContext`のインスタンス`service_context`を作成します。この`service_context`をグローバルなサービスコンテキストに設定するために、`set_global_service_context()`関数を使用します。

In [5]:
with open("../key_op.txt", "r") as f:
    key = f.readline()
openai.api_key = key
os.environ["OPENAI_API_KEY"] = key

# Chatモデル
llm = OpenAI(model="gpt-3.5-turbo")

# Embeddingモデル
# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        openai_api_key=openai.api_key
    ),
    embed_batch_size=1,
)

from llama_index import set_global_service_context

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_llm,
)

set_global_service_context(service_context)

## フォルダ内の全てのテキストファイルを読み込む

指定されたフォルダ内のすべてのテキストファイルを読み込む処理を行います。

まず、必要な拡張子を`.txt`と指定します。

次に、`SimpleDirectoryReader`クラスのインスタンスを作成します。これは、指定したディレクトリ内のテキストファイルを読み込むためのクラスです。引数として、`input_dir`に読み込むディレクトリのパス、`required_exts`に必要な拡張子のリスト、`recursive`に再帰的に読み込むかどうかを指定します。

`SimpleDirectoryReader`のインスタンスを作成した後、`load_data`メソッドを呼び出してデータを読み込みます。読み込まれたデータは、`documents`という変数に格納されます。

また、コメントアウトされている部分は別のフォルダのデータを読み込むためのコードです。`SimpleDirectoryReader`の引数にフォルダのパスを直接指定して読み込んでいます。

最後に、`documents`の要素数を表示します。

In [ ]:
# フォルダ内の全てのテキストファイルを読み込む
# https://gpt-index.readthedocs.io/en/v0.7.9/examples/data_connectors/simple_directory_reader.html
# required_exts = [".txt"]
# reader = SimpleDirectoryReader(
#     input_dir="", required_exts=required_exts, recursive=True
# )
# documents = reader.load_data()

documents = SimpleDirectoryReader(input_files=["yukkuri01.txt", "yukkuri02.txt", "yukkuri03.txt", "yukkuri04.txt"]).load_data()
print(len(documents))
documents

## テキストデータのインデックス化と保存（一旦保存した後は、この下のブロックで読み込むだけでよい）

このプログラムは、与えられたテキストデータをインデックス化し、指定されたディレクトリに保存する処理を行います。

まず、`VectorStoreIndex`のクラスの`from_documents`メソッドを使って、与えられたテキストデータをインデックス化します。`documents`は、前の部分で読み込まれたテキストデータのリストです。
次に、`index.storage_context`の`persist`メソッドを使って、インデックスデータを指定されたディレクトリに保存します。`'data/index'`は、保存先ディレクトリのパスです。このディレクトリが存在しない場合は、自動的に作成されます。

このプログラムは、テキストデータを検索する際に使用されるインデックスを作成し、保存するためのものです。

In [8]:
# index = VectorStoreIndex.from_documents(documents)

index.storage_context.persist('yukkuri_index/')

# ドキュメントも保存しておく→今回は不要（元のドキュメントを当たりたいときに使う）
# import pickle
# with open('yukkuri_index/document.txt', 'wb') as f:
#     pickle.dump(documents, f)

## インデックスの読み込み

`llama_index`モジュールから`StorageContext`と`load_index_from_storage`をインポートしています。
まず、`StorageContext`を使用してデフォルトの永続ディレクトリを設定し、`storage_context`オブジェクトを作成しています。
次に、`load_index_from_storage`関数を使用して、指定された`storage_context`からインデックスを読み込んでいます。読み込まれたインデックスは`index`変数に格納されます。

In [17]:
import pickle
from llama_index import StorageContext, load_index_from_storage
# インデックスの読み込み
storage_context = StorageContext.from_defaults(persist_dir="yukkuri_index")
index = load_index_from_storage(storage_context)

# ドキュメント読み込み
# with open('yukkuri_index/document.txt', 'rb') as f:
#     documents = pickle.load(f)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.


## 検索

このプログラムは、与えられたクエリに対して、指定された数の類似度上位の回答を取得するための処理を行います。
まず、変数`query`には、クエリ文字列"エア抜き"が格納されています。このクエリは、何らかの情報を検索・取得したい際に使用されるキーワードやフレーズとなります。

次に、変数`k`には数値3が格納されています。これは、類似度の上位k件までの回答を取得する際に使用されるパラメータです。ここでは、上位3件の回答を取得するために使用されます。

`index`という変数は、クエリエンジンのインデックスを表しているものと仮定されます。このインデックスには、検索対象となる情報や文書が格納されていると考えられます。

`query_engine`という変数には、`index`をベースに作成されたクエリエンジンが格納されます。このクエリエンジンは、与えられたクエリを元に回答を探索するための機能を提供します。また、`similarity_top_k=k`によって、類似度の上位k件までの回答を取得する設定が行われています。

最後に、`query_engine.query(query)`によって、与えられたクエリ`query`を元に、類似度の上位k件までの回答を取得します。取得された回答は、変数`answer`に格納されます。

In [30]:
query = "善良なゆっくりがひどい目に合う作品"
k=3
query_engine = index.as_query_engine(similarity_top_k=k)
answer = query_engine.query(query)

## 検索結果の表示

`answer`オブジェクトの`get_formatted_sources()`メソッドを使用して、取得された情報をフォーマットされた形式で表示します。引数として`10000`が指定されていますが、これは表示する情報の最大数を制限するためのものです。

次に、空行が表示されます。

その後、`query`変数と`answer`変数の値を表示しています。`query`変数には検索クエリが、`answer`変数には検索結果が格納されていることを示しています。

In [31]:
print(answer.get_formatted_sources(10000))
print()
print("query was:", query)
print("answer was:", answer)

> Source (Doc id: 494f2123-f3f8-47d8-b2dd-37b0926594cc): としあき(ﾕｯｸﾁ 76dc-572e):2019年09月08日 00:58 No.135497 0 0

野良ゆっくりが人間から逃れながら懸命に生きてる作品でオススメありますか？

返信： >>135500 

310.としあき(ﾗﾗﾗ 9872-abbb):2019年09月08日 02:14 No.135499 0 0

>>135494
ごめん、安価間違えたけどNo.135490もぷくーの話

311.としあき(ﾕｯｸﾁ 9b04-c762):2019年09月08日 02:55 No.135500 1 1

>>135497
それだけだとプラチナや金だけでも確実に100作以上あるからもうちょっと条件絞るか読んで面白かった話をいくつかあげたらいいんじゃないかな

312.としあき(ﾋﾞﾃｸ f4b9-403c):2019年09月14日 21:34 No.135815 0 0

孤児の子まりさメインでオススメのやつを教えてください

313.としあき(ﾃﾞｨｵｰﾝ e4da-be79):2019年09月14日 22:18 No.135816 1 0

孤児まりちゃがお題だった回のコンペに沢山あったんだけど
今コンペ期間中で検索ヒットしないんで出てこないごめん

314.としあき(ｺﾐｭｰﾝ a0a0-2f08):2019年09月14日 23:22 No.135819 2 0

番号8226〜8265を検索して出た作品の冒頭の使用したお題に【街中を必死に生きる孤児まりちゃ】があるものを読むといいと思います

315.としあき(ﾕｯｸﾁ 5ef0-79a3):2019年09月26日 00:27 No.136120 0 0

anko7244 anko11015
の様に人間の庇護から外れたゆっくりが手持ちのおもちゃなんかを切り売りしていく描写が大好きなのですが、他にも似た描写がある作品はありませんか？

316.としあき(ﾋﾞﾃｸ aab3-136f):2019年09月27日 03:45 No.136151 0 7

れいむだけを全滅させたら平和になった話を探してる
結構前に読んだんだけど
なんか途中にゴミ処理の話が入るやつ

返信： >>136156 



## 検索結果の根拠となるドキュメントを表示

この追加されたプログラムは、先ほどのプログラムの結果である`answer`から回答のIDを取得し、そのIDに対応する文書のテキストを表示する処理を行います。

まず、`ids`というリストには、`answer.source_nodes[i].node.id_`をrange(k)回繰り返して取得した回答のIDが格納されます。`answer.source_nodes[i].node.id_`は、回答の中の各ソースノードのIDを取得するための記述です。このIDは、結果に含まれる回答の中で一意の識別子となっています。

次に、`print(*ids)`によって、`ids`リストの各要素を表示します。これにより、取得した回答のIDが出力されます。

その後、`for`ループを使用して、`range(k)`回繰り返し処理を行います。これにより、`k`回分の回答を表示するための処理が行われます。

ループ内では、まず区切り線を表示するための`print("-----------------------------")`が行われます。

次に、`index.storage_context.docstore.docs[ids[i]].text`によって、`ids`リストの要素である回答のIDに対応する文書のテキストが取得されます。`index.storage_context.docstore.docs[ids[i]]`は、インデックスのドキュメントストアから、特定のIDの文書を取得するための記述です。その後、取得した文書のテキストが表示されます。

以上が、追加されたプログラムの解説です。このプログラムにより、取得した回答のIDとその回答文書のテキストが表示されることになります。



In [32]:
# 根拠となるindex（元のドキュメントを場合に応じて分割したもの）
ids = [answer.source_nodes[i].node.id_ for i in range(k)]
print(*ids)
for i in range(k):
    print("-----------------------------")
    print(index.storage_context.docstore.docs[ids[i]].text)

494f2123-f3f8-47d8-b2dd-37b0926594cc b97d414a-0a91-41c5-be8b-bd0912eb6004 5a4e3bcb-8163-4ba0-ab8f-565833846623
-----------------------------
としあき(ﾕｯｸﾁ 76dc-572e):2019年09月08日 00:58 No.135497 0 0

野良ゆっくりが人間から逃れながら懸命に生きてる作品でオススメありますか？

返信： >>135500 

310.としあき(ﾗﾗﾗ 9872-abbb):2019年09月08日 02:14 No.135499 0 0

>>135494
ごめん、安価間違えたけどNo.135490もぷくーの話

311.としあき(ﾕｯｸﾁ 9b04-c762):2019年09月08日 02:55 No.135500 1 1

>>135497
それだけだとプラチナや金だけでも確実に100作以上あるからもうちょっと条件絞るか読んで面白かった話をいくつかあげたらいいんじゃないかな

312.としあき(ﾋﾞﾃｸ f4b9-403c):2019年09月14日 21:34 No.135815 0 0

孤児の子まりさメインでオススメのやつを教えてください

313.としあき(ﾃﾞｨｵｰﾝ e4da-be79):2019年09月14日 22:18 No.135816 1 0

孤児まりちゃがお題だった回のコンペに沢山あったんだけど
今コンペ期間中で検索ヒットしないんで出てこないごめん

314.としあき(ｺﾐｭｰﾝ a0a0-2f08):2019年09月14日 23:22 No.135819 2 0

番号8226〜8265を検索して出た作品の冒頭の使用したお題に【街中を必死に生きる孤児まりちゃ】があるものを読むといいと思います

315.としあき(ﾕｯｸﾁ 5ef0-79a3):2019年09月26日 00:27 No.136120 0 0

anko7244 anko11015
の様に人間の庇護から外れたゆっくりが手持ちのおもちゃなんかを切り売りしていく描写が大好きなのですが、他にも似た描写がある作品はありませんか？

316.としあき(ﾋﾞﾃｸ aab3-136f):2019年09月27日 03:45 No

In [31]:
# 根拠となるdocument
# ids = [list(answer.source_nodes[i].node.relationships.values())[0].node_id for i in range(k)]
# print(ids)
# for document in documents:
#     if document.id_ in ids:
#         print(document)

'\n# 根拠となるdocument\nids = [list(answer.source_nodes[i].node.relationships.values())[0].node_id for i in range(k)]\nprint(ids)\nfor document in documents:\n    if document.id_ in ids:\n        print(document)\n'


## 参考

indexを更新する場合

https://dev.classmethod.jp/articles/llama-index-insert-index/

クエリの詳細をいじったりする場合等

https://zenn.dev/mganeko/scraps/181994eb7acfaf

その他参考文献

https://techblog.cccmk.co.jp/entry/2023/05/16/153732

https://dev.classmethod.jp/articles/llama-index-insert-index/

https://gpt-index.readthedocs.io/en/latest/examples/customization/llms/AzureOpenAI.html#azure-openai


In [29]:
# 検索用
import ipywidgets as widgets
from IPython.display import display

# 類似度上位いくつ持ってくるか
k = 3

# チャット履歴を表示するテキストエリア
output_area = widgets.Output(layout={'border': '1px solid black'})

# ユーザーからの入力を受け付けるテキストボックス
input_text = widgets.Text(value='', placeholder='Type something', description='Input:')

# チャットボットの返答を処理する関数
def reply(change):
    # ここでチャットボットの処理を実装します。
    if not change['new']:
        return
    with output_area:
        output_area.clear_output()  # output_areaをクリア
        message = change.new
        query_engine = index.as_query_engine(similarity_top_k=k)
        answer = query_engine.query(message)
        print("YOU:"+message+"\n\n")        
        print("GPT:"+answer.response+"\n\n")
    input_text.value = ''  # 入力フィールドをクリア

# ウィジェットを表示
display(output_area, input_text)

# テキストボックスの値が変化したときにreply関数を呼び出す
input_text.observe(reply, names='value', type='change')
input_text.continuous_update = False

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Text(value='', description='Input:', placeholder='Type something')